In [1]:
import pandas as pd
from nltk.corpus import stopwords
from stop_words import get_stop_words

In [2]:
#View Options
pd.set_option('display.max_rows',300)
pd.set_option('display.max_columns',50)

In [3]:
#Load in scraped glassdoor files
glassdoor_us = pd.read_excel(r'Data\\glassdoor_raw_usa.xlsx')

#list of skills we want frequency count for 
technical_skills = [
    "excel", "python", "r", "jupyter", "apache", "spark",
    "sas", "power", "tableau", "knime", "powerpivot", "matlab", "java",
    "javascript", "plotly", "nlp", "tensorflow", "scikit", "ggplot",
    "chartio", "oracle", "sql", "mysql", "postgresql", "pandas", "html", "bigquery",
    "hadoop", "d3", "scraping","qlik", "alteryx", "spss", "stata",
    "mongodb", "cassandra", "aws", "azure",  "looker", "redshift", "selenium", "spotfire", "hive", "impala", "kafka", "flink", "zeppelin"
]

soft_business_skills = [
    "communication", "leadership", "teamwork",
    "creativity", "adaptability",  "empathy", "organization",
    "presentation",  "innovation",
    "entrepreneurship", "collaboration",
    "crm", "speaking", "analysis",
    "marketing", "networking", "motivation",
    "research",
    "ethics", "mentoring", "sensitivity", "planning", "acumen",'visualization','statistics',"warehousing",'mining'
]

### Processing the Glassdoor USA Scraped Jobs

In [4]:
#Checking the table. Remove URL column. 
glassdoor_us.tail()

,Company,Title,Location,Salary,Industry,Revenue,Size,Description,Rating,Sector
5574,PHAXIS,Regulatory Reporting Analyst,"New York, NY",$50.00 - $60.00 Per Hour (Employer est.),NaN,NaN,NaN,We are seeking a dynamic and detail-oriented R...,NaN,NaN
5575,Truity Credit Union4.3 ★,eCommerce Business Intelligence Analyst,"Bartlesville, OK",$63K - $94K (Glassdoor est.),Banking & Lending,$25 to $100 million (USD),51 to 200 Employees,JOB SUMMARYThis position will provide business...,4.3,Financial Services
5576,CPS Energy3.7 ★,Analyst General Business 1,"San Antonio, TX",$11.00 Per Hour (Employer est.),Energy & Utilities,$1 to $5 billion (USD),1001 to 5000 Employees,"We are engineers, high line workers, power pla...",3.7,"Energy, Mining & Utilities"
5577,"TSR Consulting Services, Inc (Confidential Cli...",Business Analyst,"Seattle, WA",$79.00 - $84.00 Per Hour (Employer est.),HR Consulting,$25 to $100 million (USD),501 to 1000 Employees,Business Analyst\nA Business Analyst is needed...,4.1,Human Resources & Staffing
5578,Backcountry3.1 ★,Senior Marketing Data Analyst,"Park City, UT",$85K - $98K (Employer est.),"Department, Clothing & Shoe Stores",$500 million to $1 billion (USD),501 to 1000 Employees,Lead and take ownership of reporting on a week...,3.1,Retail & Wholesale


In [5]:
#Getting overview and removing unwanted columns and renaming others. We can see data is not uniform. Missing values should be identified and the rows removed. 
glassdoor_us.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5579 entries, 0 to 5578
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Company      5579 non-null   object 
 1   Title        5579 non-null   object 
 2   Location     5578 non-null   object 
 3   Salary       5242 non-null   object 
 4   Industry     3282 non-null   object 
 5   Revenue      3698 non-null   object 
 6   Size         3698 non-null   object 
 7   Description  4469 non-null   object 
 8   Rating       3390 non-null   float64
 9   Sector       3282 non-null   object 
dtypes: float64(1), object(9)
memory usage: 436.0+ KB


In [6]:
#Find and remove duplicates
glassdoor_us.drop_duplicates(subset=['Company','Title','Location'],inplace=True)

In [7]:
#see all null values and remove
display(glassdoor_us.isnull().sum())
glassdoor_us.dropna(inplace=True)

Company          0
Title            0
Location         1
Salary          71
Industry       216
Revenue        142
Size           142
Description     36
Rating         178
Sector         216
dtype: int64

In [8]:
#Reset index and re-check data set after removing duplicates and null values 
glassdoor_us.reset_index(drop=True,inplace=True)
glassdoor_us.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Company      627 non-null    object 
 1   Title        627 non-null    object 
 2   Location     627 non-null    object 
 3   Salary       627 non-null    object 
 4   Industry     627 non-null    object 
 5   Revenue      627 non-null    object 
 6   Size         627 non-null    object 
 7   Description  627 non-null    object 
 8   Rating       627 non-null    float64
 9   Sector       627 non-null    object 
dtypes: float64(1), object(9)
memory usage: 49.1+ KB


In [9]:
#Cleaning up reviews from the company name
glassdoor_us['Company']= glassdoor_us['Company'].str.rstrip('0123456789.★ ')
glassdoor_us.head()

,Company,Title,Location,Salary,Industry,Revenue,Size,Description,Rating,Sector
0,Harsha Autism Center,Board Certified Behavior Analyst (BCBA),"Indianapolis, IN",$87K (Employer est.),Health Care Services & Hospitals,Unknown / Non-Applicable,Unknown,Harsha Autism Center is seeking a full-time (B...,4.6,Healthcare
1,Centre for Neuro Skills,Behavior Analyst- Bakersfield,"Bakersfield, CA",$70K - $85K (Employer est.),Health Care Services & Hospitals,Unknown / Non-Applicable,501 to 1000 Employees,"Up to a $25,000 sign on bonus! *\nThe Behavior...",3.9,Healthcare
2,Aspire Learning Center,Board Certified Behavior Analyst (BCBA),"Beverly, MA",$73K - $88K (Employer est.),Preschools & Child Care Services,Unknown / Non-Applicable,Unknown,"ASPIRE LEARNING CENTER, LLC\nJob Title: Board ...",1.7,Education
3,KVC Missouri,Board Certified Behavior Analyst (BCBA),"Webster Groves, St. Louis, MO",$64K - $94K (Glassdoor est.),Civic & Social Services,$100 to $500 million (USD),1001 to 5000 Employees,The BCBA is responsible for developing and mon...,4.4,Nonprofit & NGO
4,The Genesis Group,Board Certified Behavior Analyst (BCBA),"Rio Vista, CA",$65.00 - $70.00 Per Hour (Employer est.),Information Technology Support Services,Unknown / Non-Applicable,51 to 200 Employees,Responsibilities:- Conduct assessments to iden...,4.0,Information Technology


In [10]:
#Splitting location up by city and state. Some locations have "remote" or "United states" as location, so no state, I have replaced those nulls with Remote. 
glassdoor_us[['City','State']]=glassdoor_us['Location'].str.strip().str.rsplit(',',1, expand=True)

glassdoor_us['State'].fillna('Remote',inplace=True)
glassdoor_us['State'] = glassdoor_us['State'].str.strip()

glassdoor_us[glassdoor_us['City'].str.isalpha()].head()

C:\Users\amolv\AppData\Local\Temp\ipykernel_25500\3743301021.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  glassdoor_us[['City','State']]=glassdoor_us['Location'].str.strip().str.rsplit(',',1, expand=True)


,Company,Title,Location,Salary,Industry,Revenue,Size,Description,Rating,Sector,City,State
0,Harsha Autism Center,Board Certified Behavior Analyst (BCBA),"Indianapolis, IN",$87K (Employer est.),Health Care Services & Hospitals,Unknown / Non-Applicable,Unknown,Harsha Autism Center is seeking a full-time (B...,4.6,Healthcare,Indianapolis,IN
1,Centre for Neuro Skills,Behavior Analyst- Bakersfield,"Bakersfield, CA",$70K - $85K (Employer est.),Health Care Services & Hospitals,Unknown / Non-Applicable,501 to 1000 Employees,"Up to a $25,000 sign on bonus! *\nThe Behavior...",3.9,Healthcare,Bakersfield,CA
2,Aspire Learning Center,Board Certified Behavior Analyst (BCBA),"Beverly, MA",$73K - $88K (Employer est.),Preschools & Child Care Services,Unknown / Non-Applicable,Unknown,"ASPIRE LEARNING CENTER, LLC\nJob Title: Board ...",1.7,Education,Beverly,MA
6,Applied Family Solutions,Board Certified Behavior Analyst (BCBA),"Sacramento, CA",$66K - $106K (Glassdoor est.),Primary & Secondary Schools,Unknown / Non-Applicable,1 to 50 Employees,We are looking for a BCBA to join our team. Th...,3.6,Education,Sacramento,CA
7,Pathways Autism Center,Board Certified Behavior Analyst (BCBA),"Atlanta, GA",$80K - $100K (Employer est.),Health Care Services & Hospitals,$5 to $25 million (USD),51 to 200 Employees,Pathways Autism Center is currently hiring for...,3.3,Healthcare,Atlanta,GA


In [11]:
#Cleaning Salary
#Lets get rid of ("Glassdoor or Employer Est" ) and "Per Hour". Going to stack the two slicing functions. 
glassdoor_us['Salary']=(glassdoor_us['Salary']
                        .apply(lambda x: x[:x.find('(')])
                        .apply(lambda x: x[:x.find('P')])
                        )

#combined so I can use this code quickly for all files. We start by stripping, then we remove the K and $ then we split

glassdoor_us[['min_salary','max_salary']]= (glassdoor_us['Salary']
                                            .str.replace('[$K]','',regex=True)
                                            .str.strip()
                                            .str.split('-',expand=True)
                                            )

#DIfferentiate between annual and hourly wage by creating a filter
filter_hourly_wage = glassdoor_us['Salary'].str.contains('K')

#Convert hourly wage to annual for fair comparison. Assumption of 40 Hours a week, 52 weeks of year. 

glassdoor_us['max_salary'].fillna(0.0, inplace=True)

glassdoor_us['min_salary'] = glassdoor_us['min_salary'].astype(float)
glassdoor_us['max_salary'] = glassdoor_us['max_salary'].astype(float)

glassdoor_us.loc[~filter_hourly_wage, 'min_salary'] = (glassdoor_us['min_salary'] * 40 * 52)/1000

glassdoor_us.loc[~filter_hourly_wage, 'max_salary'] = (glassdoor_us['max_salary'] * 40 * 52)/1000

glassdoor_us.loc[glassdoor_us['max_salary']==0, 'max_salary'] = glassdoor_us['min_salary']


In [12]:
#Remove Senior Roles  - 
senior_keywords = ['senior', 'lead', 'manager', 'director', 'head', 'chief', 'principal', 'vp', 'executive','sr.','sr']
glassdoor_us=glassdoor_us[~glassdoor_us['Title'].str.lower().str.contains('|'.join(senior_keywords))]


In [13]:
#Checking our changes
glassdoor_us.tail()

,Company,Title,Location,Salary,Industry,Revenue,Size,Description,Rating,Sector,City,State,min_salary,max_salary
620,GEODIS Logistics LLC,Procurement Analyst,"Brentwood, TN",$57K - $78K,Shipping & Trucking,$5 to $10 billion (USD),10000+ Employees,\nSee disclaimer 1 and 2 at the end of this Jo...,3.3,Transportation & Logistics,Brentwood,TN,57.0,78.0
621,Gallagher,Business Systems Analyst - Hybrid in Rolling M...,"Rolling Meadows, IL",$79K - $107K,Insurance Agencies & Brokerages,$1 to $5 billion (USD),10000+ Employees,This is Hybrid position (in office and work fr...,3.6,Insurance,Rolling Meadows,IL,79.0,107.0
622,Menasha Packaging,Pricing Quote Analyst,"Neenah, WI",$49K - $65K,Machinery Manufacturing,Unknown / Non-Applicable,1001 to 5000 Employees,ABOUT US (AND OUR EXCITING FUTURE)Menasha Corp...,3.2,Manufacturing,Neenah,WI,49.0,65.0
623,Nemours,Enterprise Resource Planning (ERP) Business An...,"Jacksonville, FL",$58K - $91K,Health Care Services & Hospitals,$1 to $5 billion (USD),5001 to 10000 Employees,This is a remote position.\nWe are looking for...,3.9,Healthcare,Jacksonville,FL,58.0,91.0
624,"Dovenmuehle Mortgage, Inc",Data Integration Analyst,"Lake Zurich, IL",$40K,Banking & Lending,Unknown / Non-Applicable,1001 to 5000 Employees,Data Integration Analyst\nGeneral Description:...,2.5,Financial Services,Lake Zurich,IL,40.0,40.0


### Textual Analysis

In [15]:
#Lets clean the job description of all non-alphabet chars so we can use counter to find words. Also define the stopwords. 
glassdoor_us['Description'] = glassdoor_us['Description'].str.replace('[^a-zA-Z]'," ",regex=True)
stopWords = get_stop_words('english')+stopwords.words('english')
stopWords.extend(['years','requirements','required','benefits','may','company','preferred','including','health','insurance','work','job','ensure','experience','skills','ability','support','new','position','strong','time','performance'])

In [16]:
# Replace all occurrences of the stopwords and output a list with words seperated
glassdoor_us['Description'] = glassdoor_us['Description'].str.lower().apply(lambda x: [word for word in x.split() if word not in stopWords])

#Check for the presence of technical skills in each description and store in seperate column
for skill in technical_skills+soft_business_skills:
    glassdoor_us[skill] = glassdoor_us['Description'].apply(lambda x: skill.lower() in x)

#Data viuslaization works better with narrow data
glassdoor_us_narrow = glassdoor_us.melt(id_vars=['Company', 'Title', 'Location', 'Salary', 'Industry', 'Revenue', 'Size',
       'Description', 'Rating', 'Sector', 'City', 'State', 'min_salary',
       'max_salary'], value_vars=technical_skills+soft_business_skills, var_name='Skills', value_name='Present')

glassdoor_us_narrow.head()

,Company,Title,Location,Salary,Industry,Revenue,Size,Description,Rating,Sector,City,State,min_salary,max_salary,Skills,Present
0,Harsha Autism Center,Board Certified Behavior Analyst (BCBA),"Indianapolis, IN",$87K,Health Care Services & Hospitals,Unknown / Non-Applicable,Unknown,"[harsha, autism, center, seeking, full, bcba, ...",4.6,Healthcare,Indianapolis,IN,87.0,87.0,excel,False
1,Centre for Neuro Skills,Behavior Analyst- Bakersfield,"Bakersfield, CA",$70K - $85K,Health Care Services & Hospitals,Unknown / Non-Applicable,501 to 1000 Employees,"[sign, bonus, behavior, analyst, develops, mon...",3.9,Healthcare,Bakersfield,CA,70.0,85.0,excel,False
2,Aspire Learning Center,Board Certified Behavior Analyst (BCBA),"Beverly, MA",$73K - $88K,Preschools & Child Care Services,Unknown / Non-Applicable,Unknown,"[aspire, learning, center, llc, title, board, ...",1.7,Education,Beverly,MA,73.0,88.0,excel,True
3,KVC Missouri,Board Certified Behavior Analyst (BCBA),"Webster Groves, St. Louis, MO",$64K - $94K,Civic & Social Services,$100 to $500 million (USD),1001 to 5000 Employees,"[bcba, responsible, developing, monitoring, be...",4.4,Nonprofit & NGO,"Webster Groves, St. Louis",MO,64.0,94.0,excel,False
4,The Genesis Group,Board Certified Behavior Analyst (BCBA),"Rio Vista, CA",$65.00 - $70.00,Information Technology Support Services,Unknown / Non-Applicable,51 to 200 Employees,"[responsibilities, conduct, assessments, ident...",4.0,Information Technology,Rio Vista,CA,135.2,145.6,excel,False


In [17]:
glassdoor_us.to_csv('Data\\USA Jobs Cleaned\\glassdoor_usa_final.csv',index = False)
glassdoor_us_narrow.to_csv('Data\\USA Jobs Cleaned\\glassdoor_usa_final_narrow.csv',index = False)